### This should be done after recent_tracks.csv is generated, otherwise the process takes a long time.

### This file will generate some new features in recent_tracks, specifically related to the other datasets
### 1) Whether or not the track appears in the top 200 at all
### 2) The number of streams the track gets within 1 month of its release, on the top 200 charts.
### 3) Get the artist # of followers

### This file:
### rewrites charts.csv (just adds on the track_id, which is parsed from the url)
### produces new combined_recent_tracks.csv (several columns added)
### produces new grouped_charts.csv, charts grouped by track_id, so it's easier to accumulate the stream count

In [71]:
import csv
import math
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import json
import seaborn as sns
import matplotlib.pyplot as plt
import dateutil.parser as parser

from datetime import datetime

SPRING='SPRING'
SUMMER='SUMMER'
AUTUMN='AUTUMN'
WINTER='WINTER'

SEASONS = [SPRING, SUMMER, AUTUMN, WINTER]

TRACK_IMMEDIATE_DAYS = 30 # change this if necessary

In [96]:
# Remember, we should have the following files in the top level directory:
# artists.csv, charts.csv, dict_artists.json, and tracks.csv
try:
    artists_df = pd.read_csv("./artists.csv")
    charts_df = pd.read_csv("./charts.csv")
    recent_tracks_df = pd.read_csv("./recent_tracks.csv")
except:
    pass

# this one will be generated later on.
try:
    grouped_charts_df = pd.read_csv("./grouped_charts.csv")
except:
    pass

# Overview of information

In [7]:
artists_df.tail(2)

,id,followers,genres,name,popularity
1162093,5nvjpU3Y7L6Hpe54QuvDjy,2357.0,['black comedy'],Donnell Rawlings,15
1162094,2bP2cNhNBdKXHC6AnqgyVp,40.0,['new comedy'],Gabe Kea,8


In [8]:
charts_df.head(2)

,Unnamed: 0,title,rank,date,artist,url,region,chart,trend,streams,year,track_id
0,0,Chantaje (feat. Maluma),1,2017-01-01,Shakira,https://open.spotify.com/track/6mICuAdrwEjh6Y6...,Argentina,top200,SAME_POSITION,253019.0,2017,6mICuAdrwEjh6Y6lroV2Kg
1,1,Vente Pa' Ca (feat. Maluma),2,2017-01-01,Ricky Martin,https://open.spotify.com/track/7DM4BPaS7uofFul...,Argentina,top200,MOVE_UP,223988.0,2017,7DM4BPaS7uofFul3ywMe46


In [9]:
recent_tracks_df.tail(2)

,Unnamed: 0,id,name,popularity,duration_ms,explicit,artists,id_artists,release_date,danceability,...,mode,speechiness,acousticness,instrumentalness,liveness,valence,tempo,time_signature,year,season
46667,586668,0NuWgxEp51CutD2pJoF4OM,blind,72,153293,0,['ROLE MODEL'],['1dy5WNgIKQU6ezkpZs4y8z'],2020-10-21,0.765,...,1,0.0652,0.141,0.000297,0.0924,0.6860,150.091,4,2020,AUTUMN
46668,586669,27Y1N4Q4U3EfDU5Ubw8ws2,What They'll Say About Us,70,187601,0,['FINNEAS'],['37M5pPGs6V1fchFJSgCguX'],2020-09-02,0.535,...,0,0.0408,0.895,0.000150,0.0874,0.0663,145.095,4,2020,AUTUMN


# Charts feature engineering

### IMPORTANT: these two blocks only need to be run once. in the future, just comment it out, and just read from csv.

### 1) add the track_id to the charts dataframe
### 2) make a new dataframe, grouping the charts by track_id and date. this greatly reduces the search space.
###    this way we can immediately get the number of streams a track got on any given day.

In [107]:
# IMPORTANT!
# Run this, and use it to overwrite the existing charts.csv
# def get_track_id_from_url(row):
#     url = row['url']
#     try:
#         return url.split('/')[-1]
#     except:
#         return ""
# Note: this probably takes like 5 minutes
# charts_df['track_id'] = charts_df.apply(get_track_id_from_url, axis=1)\
# charts_df.to_csv("charts.csv")

In [95]:
# grouped_charts = charts_df[['track_id','date','streams']].groupby(['track_id','date']).sum()
# grouped_charts.to_csv("grouped_charts.csv")

### Feature engineering #1: determine which songs appeared in the top 200s at least once. 

### - adds the "top_200_presence" column to the recent_tracks_df

In [113]:
top_200_track_id_set = set()

In [114]:
for track_id in charts_df['track_id'].values:
    top_200_track_id_set.add(track_id)

In [115]:
# determine if a song has appeared in the top 200s
def get_top_200_presence(row):
    track_id = row['id']
    return bool(track_id in top_200_track_id_set)

In [116]:
recent_tracks_df['top_200_presence'] = recent_tracks_df.apply(get_top_200_presence, axis=1)

In [117]:
recent_tracks_df['top_200_presence'].value_counts()

True     23884
False    22785
Name: top_200_presence, dtype: int64

### Feature engineering #2: get the total follower count for each song (based on the id_artists)
### - adds the "total_follower_count" column

In [118]:
# map of artist id to followers
artist_id_to_followers = pd.Series(artists_df.followers.values, index=artists_df.id)

In [119]:
# helper method to get number of followers by artist.
def get_followers_by_id_artist(id_artist):
    if id_artist in artist_id_to_followers:
        return artist_id_to_followers[id_artist]
    return 0

In [120]:
# takes the raw id_artists column, like "['6s1pCNXcbdtQJlsnM1hRIA', '2qGkessPTMJjSEYtxJ11KC', '3rZueITwheLTXxo6uKMdRn']"
# parses out the id's, and returns the total number of followers
def get_total_followers(row):
    id_artist_str = row["id_artists"]
    total_followers = 0
    id_artist_list = id_artist_str[1:-1].replace("'","").replace(" ", "").split(",")
    for id_artist in id_artist_list:
        total_followers += get_followers_by_id_artist(id_artist)
    return total_followers

In [122]:
recent_tracks_df['total_follower_count'] = recent_tracks_df.apply(get_total_followers, axis=1)
recent_tracks_df.head(5)

,Unnamed: 0,id,name,popularity,duration_ms,explicit,artists,id_artists,release_date,danceability,...,acousticness,instrumentalness,liveness,valence,tempo,time_signature,year,season,top_200_presence,total_follower_count
0,39511,6Pkt6qVikqPBt9bEQy8iTz,A Lover's Concerto,41,159560,0,['The Toys'],['6lH5PpuiMa5SpfjoIOlwCS'],2020-03-13,0.671,...,0.436,0.000000,0.1390,0.839,120.689,4,2020,SPRING,False,0.0
1,39529,1hx7X9cMXHWJjknb9O6Ava,The September Of My Years - Live At The Sands ...,26,187333,0,['Frank Sinatra'],['1Mxqyy3pSjf8kZZL4QVxS0'],2018-05-04,0.319,...,0.887,0.000000,0.9040,0.239,117.153,3,2018,SPRING,False,4677919.0
2,39533,19oquvXf3bc65GSqtPYA5S,It Was A Very Good Year - Live At The Sands Ho...,25,236800,0,['Frank Sinatra'],['1Mxqyy3pSjf8kZZL4QVxS0'],2018-05-04,0.269,...,0.938,0.000005,0.6830,0.160,82.332,3,2018,SPRING,False,4677919.0
3,39581,55qyghODi24yaDgKBI6lx0,"The Circle Game - Live at The 2nd Fret, Philad...",18,313093,0,['Joni Mitchell'],['5hW4L92KnC6dX9t7tYM4Ve'],2020-10-30,0.644,...,0.881,0.000022,0.7980,0.441,117.072,3,2020,AUTUMN,False,971124.0
4,39583,00xemFYjQNRpOlPhVaLAHa,"Urge For Going - Live at The 2nd Fret, Philade...",18,295093,0,['Joni Mitchell'],['5hW4L92KnC6dX9t7tYM4Ve'],2020-10-30,0.627,...,0.955,0.000162,0.0986,0.299,115.864,4,2020,AUTUMN,False,971124.0


In [180]:
# just to doublecheck it works:
# some_list = "['6s1pCNXcbdtQJlsnM1hRIA', '2qGkessPTMJjSEYtxJ11KC', '3rZueITwheLTXxo6uKMdRn']"
# get_followers_by_id_artist('6s1pCNXcbdtQJlsnM1hRIA') + get_followers_by_id_artist('2qGkessPTMJjSEYtxJ11KC') + get_followers_by_id_artist('3rZueITwheLTXxo6uKMdRn')
# recent_tracks_df.query(f'id_artists == "{some_list}"')

### Feature engineering #3: get the total number of streams for each song, within 1 month of release
### - adds the "immediate_streams_count" column

In [123]:
from IPython.display import display
def inspect_data(track_id):
    display(recent_tracks_df.query(f'id == "{track_id}"'))
    display(charts_df.query(f'track_id == "{track_id}"'))
# inspect_data('7floNISpH8VF4z4459Qo18')

In [124]:
# accepts two strings, parses them, returns number of days
def get_days_between_dates(end_date, start_date):
    parsed_end_date = parser.parse(end_date)
    parsed_start_date = parser.parse(start_date)
    delta = parsed_end_date - parsed_start_date
    return delta.days

In [125]:
release_date = recent_tracks_df.query(f'id == "{cool_id}"')['release_date'].values[0]
release_date
b = track_charts[track_charts.apply(lambda x: get_days_between_dates(x['date'], release_date) < 30, axis=1)]

In [126]:
b['streams'].sum()

140012845.0

In [131]:
idx = 0
def get_immediate_track_stream_count(track_id):
    global idx
    idx += 1
    if idx % 50 == 0:
        print(idx)
    
    release_date = recent_tracks_df.query(f'id == "{track_id}"')['release_date'].values[0]
    track_charts = grouped_charts_df.query(f'track_id == "{track_id}"') # aggregate chart dates, by track_id
    # filter to variable days immediately after release
    immediate_charts = track_charts[track_charts.apply(lambda x: get_days_between_dates(x['date'], release_date) < TRACK_IMMEDIATE_DAYS, axis=1)]
    immediate_stream_count = immediate_charts['streams'].sum()
    return immediate_stream_count

In [136]:
# recent_tracks_df['immediate_track_stream_count'] = recent_tracks_df.apply(lambda x: get_immediate_track_stream_count(x['id']), axis=1) THIS ONE LITERALLY TAKES HOURS

In [130]:
recent_tracks_df.shape

(46669, 25)

In [137]:
recent_tracks_df

,Unnamed: 0,id,name,popularity,duration_ms,explicit,artists,id_artists,release_date,danceability,...,instrumentalness,liveness,valence,tempo,time_signature,year,season,top_200_presence,total_follower_count,immediate_track_stream_count
0,39511,6Pkt6qVikqPBt9bEQy8iTz,A Lover's Concerto,41,159560,0,['The Toys'],['6lH5PpuiMa5SpfjoIOlwCS'],2020-03-13,0.671,...,0.000000,0.1390,0.8390,120.689,4,2020,SPRING,False,0.0,0.0
1,39529,1hx7X9cMXHWJjknb9O6Ava,The September Of My Years - Live At The Sands ...,26,187333,0,['Frank Sinatra'],['1Mxqyy3pSjf8kZZL4QVxS0'],2018-05-04,0.319,...,0.000000,0.9040,0.2390,117.153,3,2018,SPRING,False,4677919.0,0.0
2,39533,19oquvXf3bc65GSqtPYA5S,It Was A Very Good Year - Live At The Sands Ho...,25,236800,0,['Frank Sinatra'],['1Mxqyy3pSjf8kZZL4QVxS0'],2018-05-04,0.269,...,0.000005,0.6830,0.1600,82.332,3,2018,SPRING,False,4677919.0,0.0
3,39581,55qyghODi24yaDgKBI6lx0,"The Circle Game - Live at The 2nd Fret, Philad...",18,313093,0,['Joni Mitchell'],['5hW4L92KnC6dX9t7tYM4Ve'],2020-10-30,0.644,...,0.000022,0.7980,0.4410,117.072,3,2020,AUTUMN,False,971124.0,0.0
4,39583,00xemFYjQNRpOlPhVaLAHa,"Urge For Going - Live at The 2nd Fret, Philade...",18,295093,0,['Joni Mitchell'],['5hW4L92KnC6dX9t7tYM4Ve'],2020-10-30,0.627,...,0.000162,0.0986,0.2990,115.864,4,2020,AUTUMN,False,971124.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
46664,586665,0SjsIzJkZfDU7wlcdklEFR,John Brown's Song,66,185250,0,['Gregory Oberle'],['4MxqhahGRT4BPz1PilXGeu'],2020-03-20,0.562,...,0.961000,0.1110,0.3860,63.696,3,2020,SPRING,False,91.0,0.0
46665,586666,1ZwZsVZUiyFwIHMNpI3ERt,Skyscraper,4,106002,0,['Emilie Chin'],['4USdOnfLczwUglA3TrdHs2'],2020-02-08,0.626,...,0.856000,0.1040,0.2150,120.113,4,2020,WINTER,False,0.0,0.0
46666,586667,5rgu12WBIHQtvej2MdHSH0,云与海,50,258267,0,['阿YueYue'],['1QLBXKM5GCpyQQSVMNZqrZ'],2020-09-26,0.560,...,0.000000,0.0648,0.2110,131.896,4,2020,AUTUMN,True,896.0,0.0
46667,586668,0NuWgxEp51CutD2pJoF4OM,blind,72,153293,0,['ROLE MODEL'],['1dy5WNgIKQU6ezkpZs4y8z'],2020-10-21,0.765,...,0.000297,0.0924,0.6860,150.091,4,2020,AUTUMN,False,245944.0,0.0


In [135]:
recent_tracks_df.to_csv("./combined_recent_tracks.csv")

In [140]:
# inspect_data('5rgu12WBIHQtvej2MdHSH0')